In [12]:
import os
import os.path as osp
import json
import itertools as I
import collections as C
import random

import aiofiles
import pyarrow
import pyarrow.parquet as pq
from tqdm import tqdm

from common.utils import remove_comments

In [2]:
data_root = '/home/ma-user/local_cache/m-a-p/FineLeanCorpus'

In [3]:
# data = pq.read_table(osp.join(data_root, 'train-00000-of-00001.parquet')).to_pandas().to_dict(orient="records")
509358

509358

In [4]:
# with open(osp.join(data_root, 'FineLeanCorpusv2_fixed.jsonl'), 'r') as f:
#     data_v2_fixed = [json.loads(l) for l in f.readlines()]
# len(data_v2_fixed)
223401

223401

In [5]:
with open(osp.join(data_root, 'FineLeanCorpus_v2.jsonl'), 'r') as f:
    data_v2 = [json.loads(l) for l in f.readlines()]
len(data_v2)

509358

In [6]:
C.Counter(
    d['source'] for d in data_v2
)

Counter({'AoPs': 350714,
         'DeepMath-103k': 45853,
         'NuminaMath-TIR': 45152,
         'DeepTheorem': 31409,
         'DeepScaleR': 22360,
         'DAPO-Math-17k': 8868,
         'Omni-MATH': 1181,
         'IneqMath': 1180,
         'BlueMO': 1099,
         'Multi-Source_Math_Competition': 993,
         'TAL-SCQ5K': 393,
         'OnlineMathContest': 156})

In [7]:
C.Counter(
    d['difficulty'] for d in data_v2
)

Counter({1: 120044,
         2: 113818,
         4: 69884,
         5: 67547,
         3: 49845,
         6: 41314,
         7: 36678,
         8: 8684,
         0: 835,
         9: 671,
         10: 38})

In [114]:
d = random.choice(data_v2)
print(d['statement'])
print(d['lean_code'])

Prove that if in a group of people on a beach, 50 individuals are wearing sunglasses and 35 are wearing caps, and the probability that a randomly - selected person wearing a cap is also wearing sunglasses is $\frac{2}{5}$, then the probability that a randomly - selected person wearing sunglasses is also wearing a cap is $\frac{7}{25}$.
import Mathlib


theorem probability_sunglasses_given_caps (P : ℝ) (hP : P = 2 / 5) :
    (35 * P) / 50 = 7 / 25 := by sorry


Prove that if a set of data \(x_{1},x_{2},x_{3},x_{4},x_{5}\) has a variance of \(2\) and another set of data \(ax_{1},ax_{2},ax_{3},ax_{4},ax_{5}\) (\(a > 0\)) has a standard deviation of \(2\sqrt{2}\), then \(a = 2\).


In [9]:
import_cnt = C.Counter()
open_scoped_cnt = C.Counter()
open_cnt = C.Counter()
option_cnt = C.Counter()

indices_filtered = []

for i, d in tqdm(list(enumerate(data_v2))):
    try:
        p_raw = d['lean_code']
        p = remove_comments(p_raw).strip().replace('\nlemma ', '\ntheorem ').replace('\nexample ', '\ntheorem thm_example')
        start_pos = p.find('theorem ')
        assert start_pos != -1
        intro, stmt = p[:start_pos], p[start_pos:]

        import_list = []
        open_scoped_list = []
        open_list = []
        option_list = []

        for l in intro.splitlines():
            if len(l.strip()) == 0:
                continue
            elif l.startswith('import '):
                import_list.append(l[len('import '):].strip())
                for o in l[len('import '):].strip().split():
                    import_cnt[o] += 1
            elif l.startswith('open scoped '):
                for t in l[len('open scoped '):].strip().split():
                    open_scoped_list.append(t)
                for o in l[len('open scoped '):].strip().split():
                    open_scoped_cnt[o] += 1
            elif l.startswith('open '):
                for t in l[len('open '):].strip().split():
                    open_list.append(t)
                for o in l[len('open '):].strip().split():
                    open_cnt[o] += 1
            elif l.startswith('set_option '):
                option_list.append(l[len('set_option '):].strip())
                option_cnt[l[len('set_option '):].strip()] += 1
            else:
                raise
        indices_filtered.append(i)
    except:
        pass

print(len(indices_filtered))

100%|██████████| 509358/509358 [00:18<00:00, 28094.80it/s]


502371


In [11]:
import_cnt, open_scoped_cnt, open_cnt, option_cnt

(Counter({'Mathlib': 508572,
          'Mathlib.Geometry.Euclidean.Triangle': 690,
          'Mathlib.Tactic': 16,
          'Mathlib.Geometry.Euclidean.Basic': 3,
          'Mathlib.Algebra.Ring.Basic': 1,
          'Mathlib.Data.Real.Basic': 1}),
 Counter({'BigOperators': 105855,
          'Topology': 20524,
          'Real': 9012,
          'EuclideanGeometry': 764,
          'Matrix': 30}),
 Counter({'Real': 166630,
          'Set': 59317,
          'Finset': 55117,
          'Filter': 53247,
          'Complex': 41546,
          'TopologicalSpace': 33552,
          'Topology': 32835,
          'Function': 19983,
          'Metric': 6903,
          'Nat': 5819,
          'Polynomial': 4111,
          'Matrix': 3091,
          'Fintype': 1287,
          'Ideal': 898,
          'FiniteDimensional': 757,
          'LinearMap': 756,
          'Module': 756,
          'Module.End': 756,
          'Int': 255,
          'Subgroup': 247,
          'BigOperators': 75,
          'MvPolynomia

In [115]:
from common.utils import chunk_list
import pickle

chunksize = 1024
working_root = '/home/ma-user/workspace/formal_problem_generation/data/FineLeanCorpus'


chunks = list(chunk_list(data_v2, chunksize))
all_splits = set(
        [
            int(n[len('raw_chunk_'):-len('.pkl')]) for n in os.listdir(working_root) if n.startswith('raw_chunk_') and n.endswith('.pkl')
        ]
    )
if len(all_splits) == 0:
    all_splits = []
    for base_cnt, chunk in chunks:
        with open(osp.join(working_root, f'raw_chunk_{base_cnt}.pkl'), 'wb') as f:
            pickle.dump(chunk, f, pickle.HIGHEST_PROTOCOL)
            all_splits.append(base_cnt)
    all_splits = set(all_splits)
elif len(all_splits) != len(chunks):
    print(f'len(all_splits) != 0 and len(all_splits) != len(chunks): {len(all_splits)} - {len(chunks)}')

done_splits = set(
        [
            int(n[len('done_chunk_'):-len('.pkl')]) for n in os.listdir(working_root) if n.startswith('done_chunk_') and n.endswith('.pkl')
        ]
    )
assert done_splits.issubset(all_splits)
remaining = sorted(all_splits - done_splits)
print(f'load_and_split(): Loaded {len(remaining)} remaining splits.')

load_and_split(): Loaded 498 remaining splits.
